In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import mimikit as mmk
import ipywidgets as W
import dataclasses as dtc
from typing import Optional, Callable, Any, Tuple, List, Dict
from IPython.display import display
UI = mmk.ui

cfg = mmk.TrainFreqnetConfig()
# print(cfg.serialize())
display(mmk.views.train_arm_view(mmk.TrainARMConfig()))

In [ ]:
options_style = dict(display="flex", margin="4px",
                     justify_content="space-between",
#                      justify_items="stretch",
                     align_content='stretch',
                     flex_flow='row',
                     width="75%",
#                     flex="2 0 auto"
                    )

buttons_style = dict()

data_acc = mmk.ConfigView(
            cfg.data,
            UI.Param(name="sources",
                    widget=UI.FileWidget().widget),
            UI.Param(name="sr",
              widget=W.ToggleButtons(description="Sample Rate: ", options=[16000, 22050, 32000, 44100], index=1,
                                     style=buttons_style,
                                     layout=options_style),
             ),
            UI.Param(name="n_fft", 
                  widget=W.ToggleButtons(description="FFT Size: ", options=[512, 1024, 2048, 4096], index=2, layout=options_style)),
            UI.Param(name="hop_length", 
                 widget=W.ToggleButtons(description="Overlapp: ", options=[1, 2, 4, 8], index=2, layout=options_style),
                 compute=lambda conf, ov: conf.n_fft // ov,),
            UI.Param(name="coordinate",
                  widget=W.ToggleButtons(description="Coordinates: ", options=["magnitude", "polar"], index=0, layout=options_style),
                 compute=lambda conf, co: co[:3]),
            UI.Param(name="db_path",
                    widget=W.Text(value="train.h5", description="Db Path: ", layout=options_style))
        ).as_widget(lambda children, **kwargs: W.Accordion([W.VBox(children=children)], **kwargs), titles=("Data",), selected_index=0)
display(data_acc)

In [ ]:
net_acc = mmk.ConfigView(
            cfg.network,
            UI.Param(name="core",
                    widget=mmk.ConfigView(
                        cfg.network.core,
                        UI.Param(name="blocks",
                                widget=UI.Labeled(
                                    W.Label(value="layers per block", layout=W.Layout(min_width="max-content", margin="0 0 0 auto")),
                                    W.Text(value=str(cfg.network.core.blocks)[1:-1], layout=W.Layout(width="75%",)),
                                    W.HBox(layout=W.Layout(width="100%", margin="8px 0 8px 0")),
                                ),
                                compute=lambda c, v: tuple(map(int, (s for s in v.split(",") if s not in ("", " "))))),
                        UI.Param(name="dims_dilated",
                                widget=UI.pw2_widget(
                                W.Label(value="$N$ units per layer: ", layout=W.Layout(min_width="max-content", margin="0 0 0 auto"),),
                                W.Text(value="64", layout=W.Layout(width="25%"), disabled=False),
                                W.Button(icon="plus", layout=W.Layout(width="25%")),
                                W.Button(icon="minus", layout=W.Layout(width="25%")),
                                W.HBox(layout=W.Layout(width="100%", margin="8px 0 8px 0"))),
                                compute=lambda c, v: int(v)),
                    ).as_widget(W.VBox, ))
    ).as_widget(lambda children, **kwargs: W.Accordion([W.VBox(children=children)], **kwargs), titles=("Network",), selected_index=0,
                layout=dict(width="50%"))

display(net_acc)

In [ ]:



train_view = mmk.ConfigView(
    cfg,
    # ---------------------------
    UI.Param(
        name="data",
        widget=),
    # ---------------------------
    UI.Param(
        name="network",
        widget=),
    # ---------------------------
    UI.Param(
        name="training",
        widget=)
    # Layout
    # Config Validation
)



display(*train_view.widgets)

In [ ]:
mmk.demos.freqnet.main(cfg)